In [2]:
# =========================
# IMPORTS
# =========================

import boto3
import os
from boto3.s3.transfer import TransferConfig

print("🚀 Starting AWS S3 → RunPod S2 transfer (Sapiens model)")

# =========================
# LOCAL FILE
# =========================

LOCAL_FILE_PATH = "/tmp/sapiens_model.pt2"

# =========================
# SOURCE: AWS S3
# =========================

SOURCE_BUCKET = "ai-bmi-predictor-v2"
SOURCE_KEY = (
    "image-segmentation/sapiens/"
    "sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2"
)

print("📥 Downloading Sapiens model from AWS S3")

aws_s3 = boto3.client("s3")

aws_s3.download_file(
    Bucket=SOURCE_BUCKET,
    Key=SOURCE_KEY,
    Filename=LOCAL_FILE_PATH,
)

file_size_mb = os.path.getsize(LOCAL_FILE_PATH) / (1024 ** 2)
print(f"✅ Download complete — {file_size_mb:.2f} MB")

# =========================
# DESTINATION: RUNPOD S2 (EU-RO-1)
# =========================

DEST_BUCKET = "e9tcw5eupu"
DEST_KEY = "sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2"

print("🔐 Creating RunPod S2 client (EU-RO-1)")

runpod_s3 = boto3.client(
    "s3",
    aws_access_key_id="user_37sKcYrvnk9UXaIY3B3Zr90MH0g",
    aws_secret_access_key="rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n",
    endpoint_url="https://s3api-eu-ro-1.runpod.io",
    region_name="eu-ro-1",
)

print("✅ RunPod S2 client created")

# =========================
# MULTIPART CONFIG (REQUIRED FOR 8GB+)
# =========================

transfer_config = TransferConfig(
    multipart_threshold=100 * 1024 * 1024,   # 100 MB
    multipart_chunksize=100 * 1024 * 1024,    # 100 MB
    max_concurrency=4,
    use_threads=True,
)

# =========================
# UPLOAD
# =========================

print("⬆️ Uploading Sapiens model to RunPod S2")
print(f"Destination: s3://{DEST_BUCKET}/{DEST_KEY}")

runpod_s3.upload_file(
    Filename=LOCAL_FILE_PATH,
    Bucket=DEST_BUCKET,
    Key=DEST_KEY,
    Config=transfer_config,
)

print("🎉 Upload completed successfully!")
print(f"📍 s3://{DEST_BUCKET}/{DEST_KEY}")


🚀 Starting AWS S3 → RunPod S2 transfer (Sapiens model)
📥 Downloading Sapiens model from AWS S3
✅ Download complete — 8303.27 MB
🔐 Creating RunPod S2 client (EU-RO-1)
✅ RunPod S2 client created
⬆️ Uploading Sapiens model to RunPod S2
Destination: s3://e9tcw5eupu/sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2
🎉 Upload completed successfully!
📍 s3://e9tcw5eupu/sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2


In [5]:
import boto3
from botocore.config import Config

print("🧹 Starting SAFE cleanup of models/ and sapiens/ (RunPod S2)")

# =========================
# FORCE PATH-STYLE + S3v4
# =========================

config = Config(
    region_name="eu-ro-1",
    signature_version="s3v4",
    s3={"addressing_style": "path"}
)

# =========================
# RUNPOD S2 CLIENT
# =========================

s3 = boto3.client(
    "s3",
    aws_access_key_id="user_37sKcYrvnk9UXaIY3B3Zr90MH0g",
    aws_secret_access_key="rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n",
    endpoint_url="https://s3api-eu-ro-1.runpod.io",
    config=config,
)

BUCKET = "e9tcw5eupu"
PREFIXES = ["models/", "sapiens/"]

# =========================
# DELETE OBJECTS ONE-BY-ONE
# =========================

for prefix in PREFIXES:
    print(f"🗑️ Deleting folder: {prefix}")

    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=BUCKET, Prefix=prefix)

    deleted = 0

    for page in pages:
        for obj in page.get("Contents", []):
            key = obj["Key"]
            s3.delete_object(Bucket=BUCKET, Key=key)
            deleted += 1
            print(f"   ❌ Deleted: {key}")

    if deleted == 0:
        print(f"⚠️ No objects found under {prefix}")
    else:
        print(f"✅ Deleted {deleted} objects from {prefix}")

print("🎉 Cleanup completed successfully")


🧹 Starting SAFE cleanup of models/ and sapiens/ (RunPod S2)
🗑️ Deleting folder: models/
   ❌ Deleted: models/sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2
✅ Deleted 1 objects from models/
🗑️ Deleting folder: sapiens/
   ❌ Deleted: sapiens/.ipynb_checkpoints/Untitled-checkpoint.ipynb
   ❌ Deleted: sapiens/.ipynb_checkpoints/Untitled1-checkpoint.ipynb
   ❌ Deleted: sapiens/Untitled.ipynb
   ❌ Deleted: sapiens/Untitled1.ipynb
   ❌ Deleted: sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2
✅ Deleted 5 objects from sapiens/
🎉 Cleanup completed successfully


In [6]:
import boto3
import os
from boto3.s3.transfer import TransferConfig

print("🚀 Uploading ONLY Sapiens model file to RunPod S2")

# =========================
# LOCAL FILE (MODEL ONLY)
# =========================

LOCAL_FILE_PATH = "/tmp/sapiens_model.pt2"

# =========================
# SOURCE: AWS S3 (DOWNLOAD)
# =========================

aws_s3 = boto3.client("s3")

aws_s3.download_file(
    Bucket="ai-bmi-predictor-v2",
    Key="image-segmentation/sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2",
    Filename=LOCAL_FILE_PATH,
)

print(f"✅ Downloaded model — {os.path.getsize(LOCAL_FILE_PATH) / (1024**2):.2f} MB")

# =========================
# DESTINATION: RUNPOD S2
# =========================

runpod_s3 = boto3.client(
    "s3",
    aws_access_key_id="user_37sKcYrvnk9UXaIY3B3Zr90MH0g",
    aws_secret_access_key="rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n",
    endpoint_url="https://s3api-eu-ro-1.runpod.io",
    region_name="eu-ro-1",
)

DEST_BUCKET = "e9tcw5eupu"
DEST_KEY = "sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2"

# =========================
# MULTIPART UPLOAD (8GB SAFE)
# =========================

transfer_config = TransferConfig(
    multipart_threshold=100 * 1024 * 1024,
    multipart_chunksize=100 * 1024 * 1024,
    max_concurrency=4,
)

print("⬆️ Uploading model (no notebooks, no folders)")

runpod_s3.upload_file(
    Filename=LOCAL_FILE_PATH,
    Bucket=DEST_BUCKET,
    Key=DEST_KEY,
    Config=transfer_config,
)

print("🎉 Upload completed")
print(f"📍 s3://{DEST_BUCKET}/{DEST_KEY}")


🚀 Uploading ONLY Sapiens model file to RunPod S2
✅ Downloaded model — 8303.27 MB
⬆️ Uploading model (no notebooks, no folders)
🎉 Upload completed
📍 s3://e9tcw5eupu/sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2
